In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from google.colab import drive
import sys, os, shutil
from pathlib import Path
drive.mount("/content/drive", force_remount=True)
colabase = Path('/content/drive/My Drive/Colab Notebooks/')
course = Path('/content/drive/My Drive/course-v3/')
dlff = Path('/content/drive/My Drive/deeplearning-ff/')
#sys.path.append(str(dlff)) # cd {dlff}
os.chdir(dlff)

Mounted at /content/drive


In [0]:
#export
from exports.lg_07 import *

## Layerwise Sequential Unit Variance (LSUV)

Getting the MNIST data and a CNN

[Jump_to lesson 11 video](https://course.fast.ai/videos/?lesson=11&t=235)

In [0]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [0]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [0]:
nfs = [8,16,32,64,64]

In [0]:
#export
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    
    def forward(self, x): return self.relu(self.conv(x))
    
    @property
    def bias(self): return -self.relu.sub
    @bias.setter
    def bias(self,v): self.relu.sub = -v
    @property
    def weight(self): return self.conv.weight

In [0]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [10]:
run.fit(2, learn)

train: [2.18009921875, tensor(0.2583, device='cuda:0')]
valid: [1.16260751953125, tensor(0.5572, device='cuda:0')]
train: [0.561432890625, tensor(0.8212, device='cuda:0')]
valid: [0.2279136962890625, tensor(0.9286, device='cuda:0')]


Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [0]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [0]:
#export
def get_batch(dl, run):
    run.xb,run.yb = next(iter(dl))
    for cb in run.cbs: cb.set_runner(run)
    run('begin_batch')
    return run.xb,run.yb

In [0]:
xb,yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [0]:
#export
def find_modules(m, cond):
    """Find recursively modules satisfying a certain condition
    define condition as function, or lambda function
    >> mods = find_modules(learn.model, lambda o: isinstance(o,ConvLayer))"""
    if cond(m): return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

def is_lin_layer(l):
    """Condition function to find linear family layers"""
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

In [0]:
mods = find_modules(learn.model, lambda o: isinstance(o,ConvLayer))

In [16]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

This is a helper function to grab the mean and std of the output of a hooked layer.

In [0]:
def append_stat(hook, mod, inp, outp): #define a hook ie the f to pas to Hools(m,f)
    d = outp.data
    hook.mean,hook.std = d.mean().item(),d.std().item() # .item() to get float from tensor

In [0]:
mdl = learn.model.cuda() #pop model to gpu

So now we can look at the mean and std of the conv layers of our model.

In [19]:
with Hooks(mods, append_stat) as hooks: # hook context manager to use the hook
    mdl(xb) #run model to activate hooks
    for hook in hooks: print(hook.mean,hook.std) #hook your fruits

0.3802540898323059 0.6207240223884583
0.4048926532268524 0.6631969213485718
0.3998511731624603 0.6531000733375549
0.4170849025249481 0.6193254590034485
0.28708282113075256 0.45084869861602783


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [0]:
#export
def lsuv_module(m, xb):
    """Apply LSUV init to each individual module using a hook
    Do with one batch, adjust bias and weight till you get 0 mean, 1 stdev
    >> for m in mods: print(lsuv_module(m, xb)) """
    h = Hook(m, append_stat)
    
    while mdl(xb) is not None and abs(h.mean)  > 1e-3: 
      m.bias -= h.mean # use bias property in ConLayer
    while mdl(xb) is not None and abs(h.std-1.0) > 1e-3:
      m.weight.data /= h.std # use weight property in ConLayer

    h.remove() # dont forget to remove hook
    return h.mean,h.std 

We execute that initialization on all the conv layers in order:

In [0]:
xb,yb = get_batch(data.train_dl, run)

In [48]:
for m in mods: print(lsuv_module(m, xb))

(0.011186651885509491, 1.000272274017334)
(-0.186702162027359, 1.0000052452087402)
(-0.07696917653083801, 0.9994977116584778)
(-0.08315957337617874, 0.9999424815177917)
(-0.27443259954452515, 1.0002636909484863)


Note that the mean doesn't exactly stay at 0. since we change the standard deviation after by scaling the weight.

Then training is beginning on better grounds.

In [49]:
%time run.fit(2, learn)

train: [0.07616216796875, tensor(0.9769, device='cuda:0')]
valid: [0.07743304443359375, tensor(0.9761, device='cuda:0')]
train: [0.0500366162109375, tensor(0.9844, device='cuda:0')]
valid: [0.07615350341796875, tensor(0.9773, device='cuda:0')]
CPU times: user 2.84 s, sys: 1.2 s, total: 4.05 s
Wall time: 4.07 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [2]:
# !pip3 install fire

!python notebook2script.py 07a_lsuv.ipynb

Converted 07a_lsuv.ipynb to exports/lg_07a.py
